This file is based on ChipWhisperer-Jupyter Examples
Copyright (C) 2012-2017 NewAE Technology Inc.
Copyright (C) 2022 Stian Husum
This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.
This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.
You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.

# Setup

In [9]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
CRYPTO_TARGET = 'NONE'
FIRMWARE_PATH = 'target/simpleserial-aes-fix'

In [10]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$FIRMWARE_PATH"
cd $3
make PLATFORM=$1 CRYPTO_TARGET=$2

SS_VER set to SS_VER_1_1
rm -f -- simpleserial-aes-fix-CWLITEARM.hex
rm -f -- simpleserial-aes-fix-CWLITEARM.eep
rm -f -- simpleserial-aes-fix-CWLITEARM.cof
rm -f -- simpleserial-aes-fix-CWLITEARM.elf
rm -f -- simpleserial-aes-fix-CWLITEARM.map
rm -f -- simpleserial-aes-fix-CWLITEARM.sym
rm -f -- simpleserial-aes-fix-CWLITEARM.lss
rm -f -- objdir-CWLITEARM/*.o
rm -f -- objdir-CWLITEARM/*.lst
rm -f -- simpleserial-aes-fix.s simpleserial.s stm32f3_hal.s stm32f3_hal_lowlevel.s stm32f3_sysmem.s
rm -f -- simpleserial-aes-fix.d simpleserial.d stm32f3_hal.d stm32f3_hal_lowlevel.d stm32f3_sysmem.d
rm -f -- simpleserial-aes-fix.i simpleserial.i stm32f3_hal.i stm32f3_hal_lowlevel.i stm32f3_sysmem.i
mkdir -p .dep
.
Welcome to another exciting ChipWhisperer target build!!
arm-none-eabi-gcc (Arch Repository) 11.3.0
Copyright (C) 2021 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTI

--change-section-lma .eeprom=0 --no-change-warnings -O ihex simpleserial-aes-fix-CWLITEARM.elf simpleserial-aes-fix-CWLITEARM.eep || exit 0
.
Creating Extended Listing: simpleserial-aes-fix-CWLITEARM.lss
arm-none-eabi-objdump -h -S -z simpleserial-aes-fix-CWLITEARM.elf > simpleserial-aes-fix-CWLITEARM.lss
.
Creating Symbol Table: simpleserial-aes-fix-CWLITEARM.sym
arm-none-eabi-nm -n simpleserial-aes-fix-CWLITEARM.elf > simpleserial-aes-fix-CWLITEARM.sym
Size after:
   text	   data	    bss	    dec	    hex	filename
  12568	      8	   1784	  14360	   3818	simpleserial-aes-fix-CWLITEARM.elf
+--------------------------------------------------------
+ Default target does full rebuild each time.
+ Specify buildtarget == allquick == to avoid full rebuild
+--------------------------------------------------------
+--------------------------------------------------------
+ Built for platform CW-Lite Arm \(STM32F3\) with:
+ CRYPTO_TARGET = NONE
+ CRYPTO_OPTIONS = 
+-------------------------------

In [11]:
%matplotlib inline
import matplotlib.pyplot as plt
import chipwhisperer as cw
scope = cw.scope()
target = cw.target(scope, cw.targets.SimpleSerial)
scope.default_setup()
if "STM" in PLATFORM or PLATFORM == "CWLITEARM" or PLATFORM == "CWNANO":
    prog = cw.programmers.STM32FProgrammer
elif PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
    prog = cw.programmers.XMEGAProgrammer
else:
    prog = None

fw_path = '{}/simpleserial-aes-fix-{}.hex'.format(FIRMWARE_PATH, PLATFORM)
cw.program_target(scope, prog, fw_path)
target.output_len = 32

See https://chipwhisperer.readthedocs.io/en/latest/api.html#firmware-update


Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to program 12575 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 12575 bytes


In [13]:
scope.dis()
target.dis()

#  Test

In [12]:
from binascii import hexlify
from chipwhisperer.common.utils.util import hexStrToByteArray

target.simpleserial_write('k', hexStrToByteArray("00000000000000000000000000000000"))
target.simpleserial_wait_ack()

target.simpleserial_write('p', hexStrToByteArray("f34481ec3cc627bacd5dc3fb08f273e6")*2)
res = target.simpleserial_read('r', 32)
assert(res[:16] == hexStrToByteArray("0336763e966d92595a567cc9ce537f5e"))
assert(res[16:] == hexStrToByteArray("0336763e966d92595a567cc9ce537f5e"))

print("Success")

Success


# Unprofiled
## Constant P2

In [6]:
from tqdm import tnrange
import numpy as np
import time
import csv


num_traces =  500000

ktp = cw.ktp.Basic()
key = ktp.next_key()
const = hexStrToByteArray("00000000000000000000000000000000")

textk = hexlify(key)
textk = textk.decode("utf-8")

OUTPUT_PATH = f"traces/{textk}-fix-{num_traces}.csv"
print(OUTPUT_PATH)

with open(OUTPUT_PATH, 'a') as f:
    fields = ['plaintext', 'ciphertext', 'trace']
    writer = csv.DictWriter(f, fieldnames=fields,delimiter=';')
    for i in tnrange(num_traces, desc='Capturing traces'):
        pt = ktp.next_text()
        trace = cw.capture_trace(scope, target, pt+const, key=key)
        if trace is None:
            continue
        textpt = hexlify(trace.textin)
        textpt = textpt.decode("utf-8")
        textct = hexlify(trace.textout)
        textct = textct.decode("utf-8")
        writer.writerow({"plaintext":textpt,"ciphertext":textct,"trace":','.join([str(_) for _ in trace.wave])})

traces/2b7e151628aed2a6abf7158809cf4f3c-fix-500000.csv


/tmp/ipykernel_46717/83265468.py:22: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for i in tnrange(num_traces, desc='Capturing traces'):


Capturing traces:   0%|          | 0/500000 [00:00<?, ?it/s]

## P1 and P2 random

In [7]:
from tqdm import tnrange
import numpy as np
import time
import csv


num_traces =  500000

ktp = cw.ktp.Basic()
key = ktp.next_key()

textk = hexlify(key)
textk = textk.decode("utf-8")

OUTPUT_PATH = f"traces/{textk}-fix-{num_traces}-diff.csv"
print(OUTPUT_PATH)

with open(OUTPUT_PATH, 'a') as f:
    fields = ['plaintext', 'ciphertext', 'trace']
    writer = csv.DictWriter(f, fieldnames=fields,delimiter=';')
    for i in tnrange(num_traces, desc='Capturing traces'):
        pt = ktp.next_text()
        pt2 = ktp.next_text()
        trace = cw.capture_trace(scope, target, pt+pt2, key=key)
        if trace is None:
            continue
        textpt = hexlify(trace.textin)
        textpt = textpt.decode("utf-8")
        textct = hexlify(trace.textout)
        textct = textct.decode("utf-8")
        writer.writerow({"plaintext":textpt,"ciphertext":textct, "trace":','.join([str(_) for _ in trace.wave])})        

traces/2b7e151628aed2a6abf7158809cf4f3c-fix-500000-diff.csv


/tmp/ipykernel_46717/375575880.py:21: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for i in tnrange(num_traces, desc='Capturing traces'):


Capturing traces:   0%|          | 0/500000 [00:00<?, ?it/s]

## P1 equals P2

In [10]:
from tqdm import tnrange
import numpy as np
import time
import csv


num_traces = 500000

ktp = cw.ktp.Basic()
key = ktp.next_key()

textk = hexlify(key)
textk = textk.decode("utf-8")

OUTPUT_PATH = f"traces/{textk}-fix-{num_traces}-equal.csv"
print(OUTPUT_PATH)

with open(OUTPUT_PATH, 'a') as f:
    fields = ['plaintext', 'ciphertext', 'trace']
    writer = csv.DictWriter(f, fieldnames=fields,delimiter=';')
    for i in tnrange(num_traces, desc='Capturing traces'):
        pt = ktp.next_text()
        trace = cw.capture_trace(scope, target, pt+pt, key=key)
        if trace is None:
            continue
        textpt = hexlify(trace.textin)
        textpt = textpt.decode("utf-8")
        textct = hexlify(trace.textout)
        textct = textct.decode("utf-8")
        writer.writerow({"plaintext":textpt,"ciphertext":textct, "trace":','.join([str(_) for _ in trace.wave])})        

traces/2b7e151628aed2a6abf7158809cf4f3c-fix-500000-equal.csv


/tmp/ipykernel_46717/962950979.py:21: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for i in tnrange(num_traces, desc='Capturing traces'):


Capturing traces:   0%|          | 0/500000 [00:00<?, ?it/s]

# Profiled

In [13]:
from tqdm import tnrange
import numpy as np
import time
import csv


traces = []
num_traces =  50000

ktp = cw.ktp.Basic()
ktp.fixed_key = False

k1 = ktp.next_text()

const = hexStrToByteArray("00000000000000000000000000000000")

for i in tnrange(num_traces, desc='Capturing traces'):
    pt = ktp.next_text()
    key = ktp.next_key()
    trace = cw.capture_trace(scope, target, pt+const, key=key)
    if trace is None:
        continue
    traces.append(trace)

textk1 = hexlify(k1)
textk1 = textk1.decode("utf-8")

OUTPUT_PATH = f"traces/{textk1}-profile-fix-{num_traces}.csv"
print(OUTPUT_PATH)

f = open(OUTPUT_PATH, 'a')
with f:
    fields = ['key', 'plaintext', 'ciphertext', 'trace']
    writer = csv.DictWriter(f, fieldnames=fields,delimiter=';')
    for trace in traces:
        textkey = hexlify(trace.key)
        textkey = textkey.decode("utf-8")
        textpt = hexlify(trace.textin)
        textpt = textpt.decode("utf-8")
        textct = hexlify(trace.textout)
        textct = textct.decode("utf-8")
        writer.writerow({"key":textkey,"plaintext":textpt,"ciphertext":textct,"trace":','.join([str(_) for _ in trace.wave])})

/tmp/ipykernel_1615/130577763.py:15: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for i in tnrange(num_traces, desc='Capturing traces'):


Capturing traces:   0%|          | 0/50000 [00:00<?, ?it/s]

traces/e4e6607fa0c944819d01672f10f2fc99-profile-fix.csv


# Speed

In [10]:
from binascii import hexlify
from chipwhisperer.common.utils.util import hexStrToByteArray

target.simpleserial_write('k', hexStrToByteArray("00000000000000000000000000000000"))
target.simpleserial_wait_ack()

target.simpleserial_write('c', hexStrToByteArray("f34481ec3cc627bacd5dc3fb08f273e6"))
print(int.from_bytes(target.simpleserial_read('r', 4), byteorder='little'))

2708


In [11]:
from tqdm import tnrange
import numpy as np
import time
import csv

counts = []
num_traces =  1000

ktp = cw.ktp.Basic()
ktp.fixed_key = False

for i in tnrange(num_traces, desc='Capturing traces'):
    target.simpleserial_write('k', ktp.next_key())
    target.simpleserial_wait_ack()
    target.simpleserial_write('c', ktp.next_text())
    counts.append(int.from_bytes(target.simpleserial_read('r', 4), byteorder='little'))

print(sum(counts)/len(counts))

/tmp/ipykernel_1615/2231837125.py:12: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for i in tnrange(num_traces, desc='Capturing traces'):


Capturing traces:   0%|          | 0/1000 [00:00<?, ?it/s]

2708.0
